In [ ]:
import requests
%matplotlib inline
from PIL import Image
from bs4 import BeautifulSoup
from io import BytesIO
import matplotlib.pyplot as matplot

def menu():
    print("""
    Welcome to Happy Hour!
    What would you like to do:
    1. Look up the head quarters of your favorite beer?
    2. Test to see if the computer can recognize if there is a beer in the picture?
    3. Have the computer make a caption of a picture of your choosing?  
    """)
    
    
def get_HQ(beer):
    beer_url = beer.replace(" ", "+")
    url = ("https://www.beeradvocate.com/search/?q=%s&qt=beer" % beer_url)
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, "lxml")
    location_raw = soup.find("div", attrs = {'id' : "ba-content"}).findAll("span")[2]
    location_name = location_raw.text
    loc = location_name.replace("| ", "")
    return loc

def Vision_API(jpeg):
    ## Insert Subscription key
    subscription_key = "d456f87f10544191aae54b20c129473a"
    assert subscription_key
    

    #Gets all the valid information to run the API
    microsoft_api = 'https://eastus.api.cognitive.microsoft.com/vision/v1.0/analyze?visualFeatures=Description&language=en'
    #jpeg = input('put in your url --> MUST BE A JPEG: \n')
    headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
    paramaters   = {'visualFeatures': 'Categories,Description,Color'}
    data     = {'url': jpeg}

    #Calls the API and gets the raw data
    response = requests.post(microsoft_api, headers=headers, params=paramaters, json=data)
    response.raise_for_status()
    Vision_Data = response.json()
    return Vision_Data
##Lines 1-46 done by Andrew Robert


def get_conf(Vision_Data):
    conf = Vision_API(jpeg)['description']['captions'][0]['confidence']
    perc = conf * 100
    confidence = str("%.2f") % perc + "%"
    return confidence


def is_beer(Vision_Data): 
    Vision_Data= Vision_API(jpeg)
    tags = Vision_API(jpeg)['description']['tags']
    image = Image.open(BytesIO(requests.get(jpeg).content))
    matplot.imshow(image)
    ##prints the picture
    matplot.axis("off")
    
    if "beer" in tags:
        caption = "This is a picture of beer!"
        caption_printed = matplot.title(caption, size="x-large", y=-0.1)
    else: 
        caption = "This is not a beer but you should get one"
        caption_printed = matplot.title(caption, size="x-large", y=-0.1)
    
def get_Caption(Vision_Data):
    #gets the caption that the api made
    Vision_Data= Vision_API(jpeg)
    caption = Vision_Data["description"]["captions"][0]["text"].capitalize()
    image = Image.open(BytesIO(requests.get(jpeg).content))
    matplot.imshow(image)
    ##prints the picture
    matplot.axis("off")
    caption_printed = matplot.title(caption, size="x-large", y=-0.1)
##lines 50-80 done by Max Boen 
    


menu()
ans = input(">> ")
if ans == "1":
    beer= input("type in a beer \n")
    try:
        print("%s headquarters are in %s" % (beer.title(), get_HQ(beer)))
    except IndexError:
        print("sorry I could not find the headquarters for this beer. Please restart the program and try another!")
elif ans == "2":
    jpeg = input('put in your url --> MUST BE A JPEG: \n')
    try:
        is_beer(Vision_API)
        confidence = get_conf(Vision_API)
        print ("I am %s sure of this answer" % confidence)
    except:
        print("Sorry I could not process this image, check your url and try again!")
elif ans == "3":
    jpeg = input('put in your url --> MUST BE A JPEG: \n')
    try:
        get_Caption(Vision_API)
        confidence = get_conf(Vision_API)
        print ("I am %s sure of this answer" % confidence)
    except: 
        print("I'm sorry I could not process this image, check your url and try again!")
else:
    print("I'm sorry that's not a valid option, please restart the program")
##Lines 85-110 Done by Greg Marlo